In [1]:
import cv2 as cv
import numpy as np

In [2]:
face_c = cv.CascadeClassifier("haarcascade_frontalface_default.xml")

In [2]:

cap = cv.VideoCapture("male_sample.mp4")

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('output.mp4',fourcc, 12.0, (768,432))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret is True:
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        faces = face_c.detectMultiScale(gray)
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        faces = face_c.detectMultiScale(gray)
        for(x,y,w,h) in faces:
            cv.rectangle(frame,(x,y), (x+w, y+h), (255,0,0), 2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
        #print "found " +str(len(faces)) +" face(s)"
        cv.imshow('frame', frame)
        out.write(frame)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv.destroyAllWindows()

In [15]:
cap = cv.VideoCapture(0)
id = 2
sampleN=0;
while 1:
    ret, img = cap.read()
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_c.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        sampleN=sampleN+1;
        cv.imwrite("C:/Users/Alex-/OneDrive - Arcada/Projects/Year3/Datorseende/FaceDetection/faceData/User."+str(id)+ "." +str(sampleN)+ ".jpg", gray[y:y+h, x:x+w])
        cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        cv.waitKey(100)
    cv.imshow('img',img)
    cv.waitKey(1)
    if sampleN > 500:
        break
cap.release()

cv.destroyAllWindows()

In [16]:
from PIL import Image # For face recognition we will the the LBPH Face Recognizer 
import os
#cv.create
#recognizer = cv.createLBPHFaceRecognizer()
recognizer = cv.face.LBPHFaceRecognizer_create() # pip install opencv-contrib-python
path="C:/Users/Alex-/OneDrive - Arcada/Projects/Year3/Datorseende/FaceDetection/faceData/"
def getImagesWithID(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]   
 # print image_path   
 #getImagesWithID(path)
    faces = []
    IDs = []
    for imagePath in imagePaths:      
  # Read the image and convert to grayscale
        facesImg = Image.open(imagePath).convert('L')
        faceNP = np.array(facesImg, 'uint8')
        # Get the label of the image
        ID= int(os.path.split(imagePath)[-1].split(".")[1])
         # Detect the face in the image
        faces.append(faceNP)
        IDs.append(ID)
        cv.imshow("Adding faces for traning",faceNP)
        cv.waitKey(10)
    return np.array(IDs), faces
Ids,faces  = getImagesWithID(path)
recognizer.train(faces,Ids)
recognizer.save("C:/Users/Alex-/OneDrive - Arcada/Projects/Year3/Datorseende/FaceDetection/faceREC/trainingdata.yml")
cv.destroyAllWindows()

In [5]:
cap = cv.VideoCapture(0)
rec = cv.face.LBPHFaceRecognizer_create()
rec.read("C:/Users/Alex-/OneDrive - Arcada/Projects/Year3/Datorseende/FaceDetection/faceREC/trainingdata.yml")
id=0

font = cv.FONT_HERSHEY_COMPLEX_SMALL
#font=cv.InitFont(cv.FONT_HERSHEY_COMPLEX_SMALL,5,1,0,4)
while 1:
    ret, img = cap.read()
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_c.detectMultiScale(gray, 1.5, 5)
    for (x,y,w,h) in faces:
        cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        id,conf=rec.predict(gray[y:y+h,x:x+w])
        print(conf)
        confPrecision = 65
        if id==1 and conf < confPrecision:
            id="Alexander with glasses"
        elif id==2 and conf < confPrecision:
            id="Alexander no glasses"
        else:
            id="Unknown"
        cv.putText(img,str(id),(x,y+h),font,1, 255)
    cv.imshow('img',img)
    
    if cv.waitKey(1) == ord('q'):
        break
cap.release()

cv.destroyAllWindows()

98.32618613498869
69.16178918343053
62.83030105806786
129.7293278754117
61.8529796628468
64.17035662222204
68.205546300634
126.0815491395174
67.59780733615341
86.6638686450587
100.63989970638315
